# Task for Today  

***

## Global Temperature Time Series Forecasting  

Given *time series data about temperatures in different countries*, let's try to predict the **change in average global temperature** for the next 15 years.

We will use a Facebook's Prophet model to make our predictions. 

# Getting Started

In [ ]:
import numpy as np
import pandas as pd

import plotly.express as px

from fbprophet import Prophet

In [ ]:
data = pd.read_csv('../input/temperature-change/Environment_Temperature_change_E_All_Data_NOFLAG.csv', encoding='latin-1')

In [ ]:
data

In [ ]:
data.info()

# Preprocessing

In [ ]:
def preprocess_inputs(df):
    df = df.copy()
    
    # Remove the standard deviation examples
    df = df.query("Element == 'Temperature change'")
    
    # Group countries and take their means
    df = df.groupby('Area').mean()
    
    # Use only time series data
    df = df.loc[:, 'Y1961':]
    
    # Get global mean temperature change
    df = pd.DataFrame(df.mean()).reset_index(drop=False)
    
    # Rename columns
    df.columns=['ds', 'y']
    
    # Fix year column
    df['ds'] = df['ds'].apply(lambda x: x[1:]).astype(np.int)
    
    return df

In [ ]:
time_series = preprocess_inputs(data)

In [ ]:
time_series

In [ ]:
fig = px.line(
    time_series,
    x='ds',
    y='y',
    labels={'ds': "Year", 'y': "Change in temperature (°C)"},
    title="Average Global Temperature Change Over Time"
)

fig.show()

# In-Sample Forecast

In [ ]:
time_train = time_series.iloc[:44, :].copy()
time_test = time_series.iloc[44:, :].copy()

In [ ]:
in_model = Prophet()
in_model.fit(time_train)

In [ ]:
in_forecast = in_model.predict(time_test).loc[:, ['ds', 'yhat']]
in_forecast['ds'] = in_forecast['ds'].apply(lambda x: x.year)
in_forecast

In [ ]:
in_result_df = time_series.merge(in_forecast, on='ds', how='left')

fig = px.line(
    in_result_df,
    x='ds',
    y=['y', 'yhat'],
    color_discrete_sequence=['black', 'red'],
    labels={'ds': "Year"},
    title="Average Global Temperature Change Over Time"
)

fig.show()

# Out-of-Sample Forecast

In [ ]:
future_df = pd.DataFrame(np.arange(2020, 2046), columns=['ds'])
future_df

In [ ]:
out_model = Prophet()
out_model.fit(time_series)

In [ ]:
out_forecast = out_model.predict(future_df).loc[:, ['ds', 'yhat']]
out_forecast['ds'] = out_forecast['ds'].apply(lambda x: x.year)
out_forecast

In [ ]:
out_result_df = pd.concat([time_series, out_forecast], axis=0)

fig = px.line(
    out_result_df,
    x='ds',
    y=['y', 'yhat'],
    color_discrete_sequence=['black', 'red'],
    labels={'ds': "Year"},
    title="Average Global Temperature Change Over Time"
)

fig.show()

# Data Every Day  

This notebook is featured on Data Every Day, a YouTube series where I train models on a new dataset each day.  

***

Check it out!  
https://youtu.be/ZJouexySecc